# Checking enviroment states and action sets

In [1]:
import sys
import os.path
sys.path.append("..")
from PPO_Agent_env_model_v2 import * #PPO_Agent_v2  PPO_Agent_with_Guided_AC
from ENV_DETAILS import *
from RUN_TENSORBOARD import *

events_folder = "./logs_hyper"
main("./logs_hyper")

2024-10-07 10:07:51.347346: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num devices available:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Selected port: 50951


2024-10-07 10:07:53.885525: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.10.1 at http://localhost:50951/ (Press CTRL+C to quit)


In [2]:
ENV = "Ant-v2"
SUCESS_CRITERIA_VALUE = ENV_DETAILS[ENV]["SUCESS_CRITERIA_VALUE"]
SUCESS_CRITERIA_EPOCH = ENV_DETAILS[ENV]["SUCESS_CRITERIA_EPOCH"]
EPISODES = ENV_DETAILS[ENV]["EPISODES"]

In [3]:

env = gym.make(ENV)
env


<TimeLimit<OrderEnforcing<AntEnv<Ant-v2>>>>

In [4]:
s = env.observation_space.sample()
s

array([ 8.70760421e-01,  1.29156417e+00, -2.52654686e-01, -7.04029356e-01,
       -1.92016841e-01, -2.28403631e-01, -9.88240265e-01, -1.98240335e-02,
       -1.37666011e-01,  1.52531331e-01,  1.22271855e+00,  2.29286422e-01,
        9.96697375e-01, -8.81689141e-01,  2.78486132e-01, -7.39449676e-01,
        4.78217328e-01,  9.90931282e-02,  1.11899928e+00, -7.24817972e-01,
        3.61573147e-01, -6.03622890e-02,  1.19120139e+00, -3.22074261e-02,
       -3.40348033e-01,  3.02807986e-01, -6.98451653e-01, -3.22106459e-01,
        2.61762610e-01, -1.86934968e-04, -6.89081161e-02, -1.81791615e+00,
       -7.62946035e-02, -1.26693614e+00,  8.67775519e-01, -1.16401866e+00,
       -9.42469214e-02, -8.57539928e-01, -8.78361255e-02,  1.31636263e+00,
        6.26267054e-01,  1.58642776e+00,  3.85776902e-01,  4.58174737e-01,
        1.22104371e+00, -1.04627686e+00,  1.05919816e-01, -2.20947323e-01,
       -8.23233424e-01,  1.26086582e+00, -1.26085443e+00,  1.34126558e+00,
        7.25036944e-01,  

In [5]:
env.observation_space, env.observation_space.shape

(Box(-inf, inf, (111,), float64), (111,))

In [6]:
s = env.reset()
s

array([ 0.72214307,  0.99120184, -0.07265375, -0.0723707 , -0.08368284,
        0.09251095,  0.05884427, -0.03274768, -0.0426363 ,  0.01780594,
        0.0678261 , -0.06571997, -0.05444822, -0.10631354,  0.0744088 ,
       -0.07900672,  0.05308476,  0.10282043, -0.04686528,  0.13348025,
        0.00748234,  0.04446588,  0.04169424,  0.09687196,  0.02211004,
       -0.0240676 , -0.09594564,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [7]:
s = env.reset()
env.action_space, env.action_space

(Box(-1.0, 1.0, (8,), float32), Box(-1.0, 1.0, (8,), float32))

### Training/Hyperparam run

In [8]:
TUNING_TYPE = "BAYES"
HYPERPARAM_TUNING = True
writer= "Training/fit_PPO/"

## Select tunning parameters range

In [9]:
if HYPERPARAM_TUNING: 

    dir = r"Hyperparam_range"
    project_name = "Random_search"

    tuner = kt.RandomSearch(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo_hyperparameter_range/", evaluation_epoch = env._max_episode_steps, training_steps = 600000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.99, 
                gae_min = 0.85, gae_max = 0.96, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.00001, lr_actor_crit_max = 0.001,
                #entropy_factor = 0.05,  # WAS THIS
                entropy_factor_min = 0.001, entropy_factor_max = 0.5,
                lr_model_min = 0.000001, lr_model_max =  0.001, kl_divergence_target = None,
                #dense_layers = [42,62],
                dense_min = 32, dense_max = 256, 
                environment_name=ENV, num_layers_act = 2, 
                num_layers_model = 1,
                training_epoch = 1,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= True,
                scaling_factor_reward = 0.1
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials=10, 
            directory=dir,
            project_name=project_name
        )
    tuner.search(x=[0], y=[1])

    for trials in tuner.oracle.get_best_trials(num_trials=3):
        print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

Reloading Tuner from Hyperparam_range/Random_search/tuner0.json

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
0.96              |0.91              |discount
0.89              |0.89              |gae_lambda
0.00093988        |0.00041057        |lr_actor_critic
0.00080556        |0.00044901        |lr_model
0.11734           |0.022262          |entropy_coeff
243               |204               |dense_units_act_crit_0
226               |246               |dense_units_act_crit_1
151               |137               |n_dense_layers_model0
0.11013           |0.17381           |kl_divergence_target

Trial number :  7
Epoch: 2000 : Reward eval/Train: -76.23060105378478/-473.88834315875465 
Epoch: 4000 : Reward eval/Train: -137.64224966236281/-309.1032602769617 
Epoch: 6000 : Reward eval/Train: -131.9041929835044/-293.1177786393178 
Epoch: 8000 : Reward eval/Train: -280.93565610575797/-328.73646725565754 
Epoch: 10000 : Reward eval/Train: -385.84240565615875/-

## Applying tunning range

In [ ]:
if HYPERPARAM_TUNING: 

    dir = r"Hyperparam_kt_ppo"
    project_name = "keras_tunning_ppo_env_model"

    tuner = kt.BayesianOptimization(
            MyHyperModel( hyper_dir = dir+"/"+project_name,  writer = "logs_hyper/ppo/", evaluation_epoch = env._max_episode_steps, training_steps = 1000000,
                sucess_criteria_epochs = SUCESS_CRITERIA_EPOCH, sucess_criteria_value= SUCESS_CRITERIA_VALUE,
                discount_min = 0.90, discount_max = 0.94, 
                #discount= 0.99,
                #gae_factor = 0.95, 
                gae_min = 0.88, gae_max = 0.90, 
                policy_clip =0.2,
                lr_actor_crit_min = 0.0002, lr_actor_crit_max = 0.001,
                #entropy_factor = 0.05, 
                entropy_factor_min = 0.01, entropy_factor_max = 0.03,
                lr_model_min = 0.0002, lr_model_max =  0.0007, kl_divergence_target = 0.17,
                #dense_layers = [128,128],
                dense_min = 128, dense_max = 256,
                environment_name=ENV, num_layers_act = 2, #max_num_layers_act = 2
                num_layers_model = 1, training_epoch = 1,
                memory_size = env._max_episode_steps, 
                normalize_reward=False, normalize_advantage= True,
                scaling_factor_reward = 0.1
                #memory_size_max= env._max_episode_steps
                ),
            objective= kt.Objective('total_train_reward', direction="max"), 
            max_trials = 20,
            # distribution_strategy= strategy,
            directory=dir,
            project_name=project_name,
            #seed=0
        )
    tuner.search(x=[0], y=[1])
else : 
    
        print("Acquiring parameters ....")

        dense_units_act_crit = [128,64]
        num_layers_actor_critic = len(dense_units_act_crit)
        num_layer_m = 1
        dense_units_model = [12]

        model = run_training(training_steps = 500000,
                            discount = 0.99, 
                            dense_units_act_crit = dense_units_act_crit,  
                            dense_units_model = dense_units_model,  
                            num_layer_a_c = num_layers_actor_critic,  
                            num_layer_m = num_layer_m, 
                            writer = writer, 
                            environment_name = ENV, 
                            return_agent = True, 
                            lr_actor_critic= 0.0001,  
                            lr_model = 0.01,
                            sucess_criteria_epochs=SUCESS_CRITERIA_EPOCH, 
                            sucess_criteria_value = SUCESS_CRITERIA_VALUE, 
                            gae_lambda = 0.95, 
                            entropy_coeff = 0.05, 
                            policy_clip = 0.2, training_epoch = 20, 
                            scaling_factor_reward = 0.1, 
                            kl_divergence_target = 0.01,
                            memory_size =  env._max_episode_steps,
                            normalize_reward=False, normalize_advantage= False, use_mlflow = False)


In [ ]:
aaaaaaaaaaaa

In [ ]:
exploration_tech = "soft"
hyperparam_combination=[]
for trials in tuner.oracle.get_best_trials(num_trials=10):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)

In [ ]:
val_env = gym.make(ENV)#, render_mode = "rgb_array"
dir = r"Hyperparam_kt_ppo"
for trials in tuner.oracle.get_best_trials(num_trials=50):
    print(f"Trial id :{trials.trial_id} | Score :{trials.score} -->",trials.hyperparameters.values)    
    training_steps = 2000000
    entropy_factor = trials.hyperparameters.values["entropy_coeff"]
    discount = trials.hyperparameters.values["discount"]
    gae = trials.hyperparameters.values["gae_lambda"]
    policy_clip = trials.hyperparameters.values["policy_clip"]
    scaling_factor_reward= trials.hyperparameters.values["scaling_factor_reward"]
    
    
    lr_actor=  trials.hyperparameters.values["lr_actor"]
    lr_critic=  trials.hyperparameters.values["lr_critic"]
    lr_model=  trials.hyperparameters.values["lr_model"]
    
    try:
        n_dense_layers_actor = trials.hyperparameters.values["n_dense_layers_actor"]
    except : 
        n_dense_layers_actor = 1
        
    try:
        n_dense_layers_critic = trials.hyperparameters.values["n_dense_layers_critic"]
    except:
        n_dense_layers_critic = 1
    

    dense_layers_actor = []
    for i in range(n_dense_layers_actor):
        dense_layers_actor.append(trials.hyperparameters.values['dense_units_act_'+str(i)])

    dense_layers_critic = []
    for i in range(n_dense_layers_critic):
        dense_layers_critic.append(trials.hyperparameters.values['dense_units_crit_'+str(i)])

    
    n_dense_layers_model = 1
    dense_layers_model = []
    for i in range(n_dense_layers_model):
        dense_layers_model.append(trials.hyperparameters.values['n_dense_layers_model'+str(i)])

    model = run_training(
        training_steps = training_steps,   
            discount = discount,
            dense_units_act = dense_layers_actor, 
            dense_units_crit = dense_layers_critic,
            dense_units_model = dense_layers_model,
            num_layer_a = n_dense_layers_actor,
            num_layer_c = n_dense_layers_critic,
            num_layer_m = n_dense_layers_model,
            writer = writer,  
            save_factor=50000, 
            sucess_criteria_epochs =SUCESS_CRITERIA_EPOCH, 
            sucess_criteria_value = SUCESS_CRITERIA_VALUE,
            environment_name=ENV,
            reward_scaler = 1, 
            evaluation_epoch = env._max_episode_steps,
            return_agent = True,
            lr_actor= lr_actor, 
            lr_critic= lr_critic,
            lr_model= lr_model,
            gae_lambda=gae,
            training_epoch= 200,
            entropy_coeff= entropy_factor,
            policy_clip = policy_clip,
            memory_size= env._max_episode_steps,
            id = int(trials.trial_id),
            scaling_factor_reward = scaling_factor_reward)
        
final_rewards = final_evaluation(model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
env_model = tuner.get_best_models()[0]
final_rewards = final_evaluation(env_model,val_env,n_tries=200, exploration=exploration_tech,  video_name = "./ppo_"+exploration_tech+"_video.mp4", env= ENV)

In [ ]:
print("Final mean reward '",exploration_tech,"':", np.mean(final_rewards))


In [ ]:
final_rewards